In [ ]:
%pip install langchain
%pip install langchain_community
%pip install anthropic_bedrock
%pip install databricks
%pip install "databricks-vector-search"

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━

---------------------------------------------------------------------------
CalledProcessError                        Traceback (most recent call last)
File <command-996910454461299>, line 5
      3 get_ipython().run_line_magic('pip', 'install anthropic_bedrock')
      4 get_ipython().run_line_magic('pip', 'install databricks')
----> 5 get_ipython().run_line_magic('pip', 'install "databricks-vector-search"')

File /databricks/python/lib/python3.10/site-packages/IPython/core/interactiveshell.py:2417, in InteractiveShell.run_line_magic(self, magic_name, line, _stack_depth)
   2415     kwargs['local_ns'] = self.get_local_scope(stack_depth)
   2416 with self.builtin_trap:
-> 2417     result = fn(*args, **kwargs)
   2419 # The code below prevents the output from being displayed
   2420 # when using magics with decodator @output_can_be_silenced
   2421 # when the last Python token in the expression is a ';'.
   2422 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks

In [ ]:
#This script uses the bedrock api to connect to the claude instant llm inorder to assign a sentiment score to each row in our data set.

from io import StringIO
import databricks
import langchain_community
import langchain
import os
import boto3
import botocore
import anthropic_bedrock
from anthropic_bedrock import AnthropicBedrock
from langchain_community.retrievers import AmazonKendraRetriever
from databricks.vector_search.client import VectorSearchClient
from langchain.vectorstores import DatabricksVectorSearch
from langchain.embeddings import DatabricksEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.chains import RetrievalQA
from botocore.config import Config

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
import logging
from pyspark.sql import Row

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType

import re

from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id


kendra_index = '' # insert your kendra index here
bedrock_region = '' # aws region
kendra_region = '' # kendra index region
user_key = '' # user key
user_scrt_key = '' # secret key

def get_kendra_doc_retriever():

    #my_config = Config(region_name = kendra_region)
    kendra_client = boto3.client("kendra", kendra_region,
                                 aws_access_key_id=user_key,
                                 aws_secret_access_key=user_scrt_key)
    retriever = AmazonKendraRetriever(index_id=kendra_index, top_k=3, client=kendra_client, attribute_filter={
        'EqualsTo': {
            'Key': '_language_code',
            'Value': {'StringValue': 'en'}
        }
    })
    return retriever

def format_prompt(query):
    return f"\n\nHuman: {query} \n\nAssistant:"

def custom_function(value):
    formatted_query = format_prompt("Please give me a numeric sentiment score between -5 and 5, in the form sentiment=x, for the following text: "+value)

    try:
        retriever = get_kendra_doc_retriever()

        bedrock_client = boto3.client("bedrock-runtime", bedrock_region,
                                        aws_access_key_id=user_key,
                                        aws_secret_access_key=user_scrt_key)

        llm = Bedrock(model_id="anthropic.claude-instant-v1", region_name=bedrock_region,
                    client=bedrock_client,
                    model_kwargs={"max_tokens_to_sample": 1000, "temperature": 1.0})


        qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

        response = qa(formatted_query)
    except:
        response="fail"

    return str(response)

def custom_parse_function(value):
    sentiment="0"
    m="0"
    pattern = r"sentiment\s*=\s*(-?\d+)"
    match = re.search(pattern, str(value))

    # Extract the value of sentiment if there is a match
    if match:
        sentiment = match.group(1)
        m="1"
    return int(sentiment)


# Register the UDF
custom_udf = udf(custom_function, StringType())
parse_udf  = udf(custom_parse_function, StringType())

df = spark.sql(f"SELECT *, NULL as sentiment_text, 0 as sentiment_value FROM default.enronConversations limit 5000")
#df.display()
df = df.withColumn("sentiment_text", custom_udf(df["content"]))

window_spec = Window.orderBy(monotonically_increasing_id())
df = df.withColumn("row_number", monotonically_increasing_id())
df = df.drop("row_num")
df = df.withColumn("sentiment_value", parse_udf(df["sentiment_text"]))
df.display()




java.lang.IllegalStateException: jupyter client is not available because the python kernel is not defined. The kernel may be restarting or the repl may have been shut down.
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.$anonfun$getJupyterKernelListener$1(JupyterDriverLocal.scala:314)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.com$databricks$backend$daemon$driver$JupyterDriverLocal$$getJupyterKernelListener(JupyterDriverLocal.scala:313)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.executePython(JupyterDriverLocal.scala:1053)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:941)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$35(DriverLocal.scala:1103)
	at com.databricks.unity.UCSEphemeralState$Handle.runWith(UCSEphemeralState.scala:45)
	at com.databricks.unity.HandleImpl.runWith(UCSHandle.scala:103)
	at com.databricks.backend.daemon.dr

In [ ]:
%sql
select content, sentiment_value from default.enronemailssentiment limit 5

java.lang.IllegalStateException: jupyter client is not available because the python kernel is not defined. The kernel may be restarting or the repl may have been shut down.
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.$anonfun$getJupyterKernelListener$1(JupyterDriverLocal.scala:314)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.com$databricks$backend$daemon$driver$JupyterDriverLocal$$getJupyterKernelListener(JupyterDriverLocal.scala:313)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.executePython(JupyterDriverLocal.scala:1053)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:941)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$35(DriverLocal.scala:1103)
	at com.databricks.unity.UCSEphemeralState$Handle.runWith(UCSEphemeralState.scala:45)
	at com.databricks.unity.HandleImpl.runWith(UCSHandle.scala:103)
	at com.databricks.backend.daemon.dr

In [ ]:
from databricks.vector_search.client import VectorSearchClient
import mlflow.deployments

dbclient = mlflow.deployments.get_deploy_client("databricks")

client = VectorSearchClient()

#Our Vector Search Endpoint
print(client.list_endpoints())
#Our Vector Search Index
print(client.list_indexes("embeddings"))
#Our Embedding and External Model(aws bedrock) Endpoint
print(dbclient.list_endpoints())

java.lang.IllegalStateException: jupyter client is not available because the python kernel is not defined. The kernel may be restarting or the repl may have been shut down.
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.$anonfun$getJupyterKernelListener$1(JupyterDriverLocal.scala:314)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.com$databricks$backend$daemon$driver$JupyterDriverLocal$$getJupyterKernelListener(JupyterDriverLocal.scala:313)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.executePython(JupyterDriverLocal.scala:1053)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:941)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$35(DriverLocal.scala:1103)
	at com.databricks.unity.UCSEphemeralState$Handle.runWith(UCSEphemeralState.scala:45)
	at com.databricks.unity.HandleImpl.runWith(UCSHandle.scala:103)
	at com.databricks.backend.daemon.dr